***Introduction***

This document provides a detailed overview of the methods and calculations essential for converting raw sensor currents into accurate gas concentration readings using Alphasense sensors. As leaders in gas detection technology, Alphasense provides robust solutions that are crucial for a variety of applications ranging from environmental monitoring to industrial safety. The focus here is to elucidate the specific equations and algorithms that are fundamental to the sensor data interpretation process, enabling users to transform electrical signals emitted by the sensors into meaningful data.

Contained within, you will find a comprehensive exploration of the mathematical models and conversion formulas as provided by Alphasense, designed to facilitate the accurate calculation of gas concentrations from the raw currents measured by the sensors. This documentation is intended to be a valuable resource for engineers, researchers, and professionals who rely on precise and dependable gas measurement techniques. Through an in-depth understanding of these conversion principles, users can ensure the effective application of Alphasense technology in their respective fields, thereby enhancing the reliability and accuracy of their gas detection systems.

***Terms definitions***

- WEu = uncorrected raw WE output
- AEu = uncorrected raw AE output
- WEc = corrected WE output
- WEe = WE electronic offset on the AFE or ISB
- AEe = AE electronic offset on the AFE or ISB
- WEo = WE sensor zero, i.e. the sensor WE output in zero air
- AEo = AE sensor zero, i.e. the sensor AE output in zero air
- WET = Total WE zero offset
- AET = Total AE zero offset

- nT = temperature dependent correction factor for algorithm 1, refer to Table 3 for values
- kT = temperature dependent correction factor for algorithm 2, refer to Table 3 for values
- k'T = temperature dependent correction factor for algorithm 3, refer to Table 3 for values
- k''T = temperature dependent correction factor for algorithm 4, refer to Table 3 for values


***Algorithms***

- Algorithm 1
    ```
    WEc = (WEu - WEe) - nT * (AEu - AEe)
    ```
- Algorithm 2
    ```
    WEc = (WEu - WEe) - kT * (WEo / AEo) * (AEu - AEe)
    ```
- Algorithm 3
    ```
    WEc = (WEu - WEe) - (WEo - AEo) - k'T * (AEu - AEe)
    ```
- Algorithm 4
    ```
    WEc = (WEu - WEe) - WEo - k''T
    ```

***Sensorbee Sensors***

- CO-B4
    - Type: B
    - Recommended Algorithm: Algorithm 1

- H2S-B4
    - Type: B
    - Recommended Algorithm: 1

- NO-B4
    - Type: B
    - Recommended Algorithm: 1

- NO2-B43F
    - Type: B
    - Recommended Algorithm: 1

- OX-B431
    - Type: B
    - Recommended Algorithm: 1

- SO2-B4
    - Type: B
    - Recommended Algorithm: 4


In [10]:
const gasTypes = {
    'CO-B4': {
        algo: 1
    }, 
    'H2S-B4': {
        algo: 1
    }, 
    'NO-B4': {
        algo: 1
    },
    'NO2-B43F': {
        algo: 1
    },
    'O3-B431': {
        algo: 1
    },
    'SO2-B4': {
        algo: 4
    }
};


const convertToTargetGas = (type, parameters) => {
    const { algo } = gasTypes[type];
    
};




SyntaxError: invalid syntax (3752340391.py, line 23)